The socketserver module is a framework for creating network servers. It defines classes for handling synchronous network requests (the server request handler blocks until the request is completed) over TCP, UDP, Unix streams, and Unix datagrams. It also provides mix-in classes for easily converting servers to use a separate thread or process for each request.

In [8]:
import logging
import sys
import socketserver

logging.basicConfig(level=logging.DEBUG, format='%(name)s : %(message)s')

class EchoRequestHandler(socketserver.BaseRequestHandler):
    def __init__(self, request, client_address, server):
        self.logger = logging.getLogger('EchoRequestHandler')
        self.logger.debug('__init__')
        super().__init__(request, client_address, server)
    def setup(self):
        self.logger.debug('setup')
        super().setup()
    def handle(self):
        self.logger.debug('handle')
        data = self.request.recv(1024)
        self.logger.debug('receive --> %s', data)
        self.request.send(data)
    def finish(self):
        self.logger.debug('finish')
        return super().finish()

In [11]:
class EchoServer(socketserver.TCPServer):
    def __init__(self, server_address, handler_class=EchoRequestHandler):
        self.logger = logging.getLogger('EchoServer')
        self.logger.debug('__init__')
        super().__init__(server_address, handler_class)
    def server_activate(self):
        self.logger.debug('server_activate')
        super().server_activate()
    def serve_forever(self, poll_interval=0.5):
        self.logger.debug('server_forever, waiting for request')
        self.logger.info('Handing requests, press <ctrl-c> to quit')
        super().serve_forever(poll_interval)
    def handle_request(self):
        self.logger.debug('handle_request')
        return super().handle_request()
    def verify_request(self, request, client_address):
        self.logger.debug(f'verify_request({request} {client_address})')
        return super().verify_request(request, client_address)
    def process_request(self, request, client_address):
        self.logger.debug(f'process_request({request} {client_address})')
        return super().process_request(request, client_address)
    def server_close(self):
        self.logger.debug('server_close')
        return super().server_close()
    def finish_request(self, request, client_address):
        self.logger.debug(f'finish_request({request} {client_address})')
        return super().finish_request(request, client_address)
    def close_request(self, request_address):
        self.logger.debug(f'close_request({request_address})')
        return super().close_request(request_address)
    def shutdown(self):
        self.logger.debug('shutdown')
        return super().shutdown()

In [12]:
import threading
import socket

address = ('localhost', 0) # let the kernel assign a port
server = EchoServer(address, EchoRequestHandler)
ip, port = server.server_address

# start the server in a thread
t = threading.Thread(target=server.serve_forever)
t.setDaemon(True)
t.start() 

logger = logging.getLogger('client')
logger.info(f'Server on {ip} {port}')
logger.debug('creating socket')
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((ip, port))

# sending data
message = 'hello world '.encode()
logger.debug('sending data : %r', message)
len_sent = s.send(message)

# receive res
response = s.recv(len_sent)
logger.debug('response from server: %r', response)

server.shutdown()
s.close()
server.socket.close()

EchoServer : __init__
EchoServer : server_activate
EchoServer : server_forever, waiting for request
client : Server on 127.0.0.1 53867
EchoServer : Handing requests, press <ctrl-c> to quit
client : creating socket
EchoServer : verify_request(<socket.socket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 53867), raddr=('127.0.0.1', 40308)> ('127.0.0.1', 40308))
client : sending data : b'hello world '
EchoServer : process_request(<socket.socket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 53867), raddr=('127.0.0.1', 40308)> ('127.0.0.1', 40308))
EchoServer : finish_request(<socket.socket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 53867), raddr=('127.0.0.1', 40308)> ('127.0.0.1', 40308))
EchoRequestHandler : __init__
EchoRequestHandler : setup
EchoRequestHandler : handle
EchoRequestHandler : receive --> b'hello world '
client : response from 

In [18]:
class ThreadedEchoRequestHandler(socketserver.BaseRequestHandler):
    def handle(self):
        # echo the back to the client
        data = self.request.recv(1024)
        cur_thread = threading.currentThread()
        res = b'%s:%s' % (cur_thread.getName().encode(), data)
        self.request.send(res)
        
class ThreadedEchoServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass


In [19]:
address = ('localhost', 0) # let the kernel assign a port
server = ThreadedEchoServer(address, ThreadedEchoRequestHandler)
ip, port = server.server_address

# start the server in a thread
t = threading.Thread(target=server.serve_forever)
t.setDaemon(True)
t.start()
print('server runing on ', t.getName())


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((ip, port))

# sending data
message = 'hello world '.encode()
print('sending data : ', message)
len_sent = s.send(message)

# receive res
response = s.recv(1024)
print('response from server: ', response)

server.shutdown()
s.close()
server.socket.close()

server runing on  Thread-15
sending data :  b'hello world '
response from server:  b'Thread-16:hello world '


In [20]:
import os
class ForkingEchoRequestHandler(socketserver.BaseRequestHandler):
    def handle(self):
        # echo the back to the client
        data = self.request.recv(1024)
        cur_pid = os.getpid()
        res = b'%d:%s' % (cur_pid, data)
        self.request.send(res)
        
class ForkingEchoServer(socketserver.ForkingMixIn, socketserver.TCPServer):
    pass

In [21]:
address = ('localhost', 0) # let the kernel assign a port
server = ForkingEchoServer(address, ForkingEchoRequestHandler)
ip, port = server.server_address

# start the server in a thread
t = threading.Thread(target=server.serve_forever)
t.setDaemon(True)
t.start()
print('server runing on ', os.getpid())


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((ip, port))

# sending data
message = 'hello world '.encode()
print('sending data : ', message)
len_sent = s.send(message)

# receive res
response = s.recv(1024)
print('response from server: ', response)

server.shutdown()
s.close()
server.socket.close()

server runing on  2805
sending data :  b'hello world '
response from server:  b'4043:hello world '
